In [1]:
from main import *
import os
import pickle
import torch
from torchinfo import summary

data_dir = '/home/cwlin/explainable_credit/data/edge_file'
cluster_setting = 'kmeans'
n_cluster = 100
inter_gat_hidn_dim=4
intra_gat_hidn_dim=4
lstm_hidn_dim=64
device = torch.device('cuda:0')
inner_edge_idx = torch.load(os.path.join(data_dir, f'{cluster_setting}_cluster_{n_cluster}', 'inner_edge_idx.pt'), map_location=device)
outer_edge_idx = torch.load(os.path.join(data_dir, f'{cluster_setting}_cluster_{n_cluster}', 'outer_edge_idx.pt'), map_location=device)
with open(os.path.join(f'{data_dir}/{cluster_setting}_cluster_{n_cluster}', 'company_to_sector_idx.pkl'), 'rb') as f:
    company_to_sector_idx = pickle.load(f)

model = CategoricalGraphAtt(14, 12, 8+1, lstm_hidn_dim, inter_gat_hidn_dim, intra_gat_hidn_dim, inner_edge_idx, outer_edge_idx, company_to_sector_idx, device)
model_path = f'/home/ybtu/codes.credit.relation.dev/NeuDP_GAT/experiments/index/{cluster_setting}_{n_cluster}/NeuDP_GAT_12_index_lstm{lstm_hidn_dim}_intra{intra_gat_hidn_dim}_inter{inter_gat_hidn_dim}/last_weights/NeuDP_GAT_checkpoint_{n_cluster}.pt'

model.load_state_dict(torch.load(model_path))

model.to(device)

# num_stock = len(pd.read_csv('/home/cwlin/explainable_credit/data/all_company_ids.csv', index_col=0).id.unique())
ts_input = torch.randn(12, 15786, 14)

summary(model, input_data=ts_input)

/home/cwlin/miniconda3/envs/gnn4rec/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Shape of daily_data_batch torch.Size([12, 15786, 14])


RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

In [ ]:
# Function to print model with hierarchy and parameters
def print_model_with_hierarchy(model, indent=""):
    total_params = 0
    for name, module in model.named_children():
        print(f"{indent}{name}:")
        if list(module.children()):
            child_params = print_model_with_hierarchy(module, indent + "  ")
            total_params += child_params
        else:
            for param_name, param in module.named_parameters():
                if param.requires_grad:
                    param_size = param.numel()
                    total_params += param_size
                    print(f"{indent}  {param_name}: Parameters: {param_size}")
    return total_params

# Print the model architecture with hierarchy and parameters
# total_parameters = print_model_with_hierarchy(model)
# print(f"Total Parameters: {total_parameters}")

bn:
  weight: Parameters: 168
  bias: Parameters: 168
sequence_encoder:
  gru_cell:
    x2h:
      W: Parameters: 2688
      b: Parameters: 192
    h2h:
      W: Parameters: 12288
      b: Parameters: 192
intra_gat:
  lin_src:
    weight: Parameters: 256
cat_gat:
  lin_src:
    weight: Parameters: 16
fusion:
  weight: Parameters: 4608
  bias: Parameters: 64
logit_f:
  weight: Parameters: 576
  bias: Parameters: 9
Total Parameters: 21225
